In [ ]:
# Pandas er superviktig og det vi bruker stort sett nedover i arket
import pandas as pd
# Vi trenger dette biblioteket for å håndtere objektet vi får tilbake fra statbanken 
from pyjstat import pyjstat
# Vi bruker requests for å spørre statbanken om svar
import requests

#### 06326: Medlemmer i trus- og livssynssamfunn som mottek offentleg stønad og er utanfor Den norske kyrkja, etter religion/livssyn
https://www.ssb.no/statbank/table/06326/

In [ ]:
# Hent dette fra api-spørringen under den genererte statbanken

# Adressen til statbanktabellen
URL = 'http://data.ssb.no/api/v0/no/table/06326'
# Begrensningene og det vi etterspør på denne tabellen, dette går inn i en "json-header-payload"
payload = {
  "query": [
    {
      "code": "ReligionLivs",
      "selection": {
        "filter": "item",
        "values": [
          "999",
          "800",
          "200",
          "300",
          "400",
          "500",
          "600",
          "700",
          "900",
          "901"
        ]
      }
    }
  ],
  "response": {
    "format": "json-stat2"
  }
}

In [ ]:
# Spør APIet om å få resultatet med requests-biblioteket
resultat = requests.post(URL, json = payload)

In [ ]:

# Putt teksten i resultatet inn i ett pyjstat-datasett-objekt
dataset = pyjstat.Dataset.read(resultat.text)
# Skriv pyjstat-objektet ut som en pandas dataframe
df = dataset.write('dataframe')
# Se på dataframen
df

In [ ]:
# Måter å se på datasettet, uncomment de linjene du vil prøve ut

# Beskrivende funksjoner
#df.info()
#df.describe()
#df['år'].value_counts()

# Kolonnenavn
#df.keys()
#df.columns

# Kolonne datatyper
#df.dtypes

# Data du kanskje vil bruke i koden din
#df.shape[1]
#len(df)

In [ ]:
# Konverter år til heltall
df['år'] = df['år'].astype(int)
df.dtypes

In [ ]:
# Dropp kolonnen som er lik på hver linje
df.drop('statistikkvariabel', axis = "columns", inplace = True)

In [ ]:
df

In [ ]:
# Kutt ut oppsummeringsvariabelen
df = df[~df['religion/livssyn'].isin(['Medlemmer i alt'])]


In [ ]:
# Døp om kolonner
df.columns = ['religion', 'aar', 'value']
df

In [ ]:
# Måter å gjøre utdrag fra datasettet
# For at dette skal ha noen konsekvens så må du assigne dem til en variabel.
# F.eks. 
#df_2020 = df[df['år'] == 2020]


# Radene hvor året er 2020
#df[df['aar'] == 2020]
# Totalt antall medlemmer i 2020 utenfor den norske kirke
#df[df['aar'] == 2020]['value'].sum()


# Radene med Kristendom, men kun returner "value"-kolonnen
#df[df['religion'] == 'Kristendom']['value']
# Sjekk gjennomsnittlig medlemstall i Kristendom 2006-2020
#df[df['religion'] == 'Kristendom']['value'].mean()

# Rader hvor året er etter 2010 og medlemmer er over 10 000
#df.loc[(df['aar'] >= 2010) & (df['value'] >= 10000)]

# Tredje raden
#df['aar'].iloc[0]
# De to siste radene
#df.iloc[-5:]
# Alle radene baklengs
#df.iloc[::-1]

In [ ]:
# Høyeste året i datasettet
max_aar = df['aar'].max()
max_aar

In [ ]:
# Radene som inneholder det høyeste året i årskolonnen
df[df['aar'] == max_aar]

In [ ]:
df_thisyr = df[df['aar'] == max_aar]
# Sorter gruppene etter verdien på det siste forekommende året
df_thisyr = df_thisyr.sort_values('value', ascending = False)

print(f'Størrelser for {max_aar}')
df_thisyr

In [ ]:
# Pivot til ett format matplotlib forventer
df_piv = df.pivot('aar', 'religion', 'value')
df_piv

In [ ]:
# Kolonnene kommer nå alfabetisk etter pivot, så vi sorterer etter motsatt størrelse i 2020
df_piv = df_piv[list(df_thisyr['religion'].iloc[::-1])]
df_piv

In [ ]:
# Output ett stacked area-plot for å se på størrelsene
df_piv.plot.area(figsize = (15,10))